# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 716 ms, sys: 269 ms, total: 985 ms
Wall time: 588 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.48 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_180_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_180_6561"
random_state = 11


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/rio/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1180980 entries, 0 to 1180979
Columns: 188 entries, id to feat179
dtypes: float64(183), int32(3), object(2)
memory usage: 1.6+ GB
CPU times: user 17.8 s, sys: 1.02 s, total: 18.8 s
Wall time: 19.7 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244892,-0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,0.300844,...,-0.080274,-0.115552,-0.174807,0.244872,0.319478,-0.059064,-0.000568,-0.244892,-0.300844,0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.146579,0.043053,...,0.187020,0.091249,-0.074116,0.114272,-0.147971,-0.194939,0.194123,0.055547,-0.043053,0.146579
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.013027,-0.001277,...,-0.007714,0.018139,-0.013208,-0.022654,0.017319,0.006334,-0.006175,0.006953,0.001277,-0.013027
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.003440,0.011453,...,-0.015015,-0.019046,0.015313,0.003978,-0.008323,0.008902,-0.003899,-0.013061,0.011453,0.003440


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  180
n_hamiltonians:  6561
n_ts:  2
CPU times: user 438 µs, sys: 31 µs, total: 469 µs
Wall time: 284 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.5450388660265203
% val:  0.0
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  3576
number of val hamiltonians:  0
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/5 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:  20%|██        | 1/5 [00:53<03:34, 53.61s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:  40%|████      | 2/5 [01:47<02:40, 53.66s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 5/5 [04:26<00:00, 53.15s/it]

running experiments: 100%|██████████| 5/5 [04:26<00:00, 53.35s/it]

CPU times: user 29min 27s, sys: 1.32 s, total: 29min 29s
Wall time: 4min 26s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244892,-0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.130846,0.300844,...,-0.080274,-0.115552,-0.174807,0.244872,0.319478,-0.059064,-0.000568,-0.244892,-0.300844,0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.146579,0.043053,...,0.187020,0.091249,-0.074116,0.114272,-0.147971,-0.194939,0.194123,0.055547,-0.043053,0.146579
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.013027,-0.001277,...,-0.007714,0.018139,-0.013208,-0.022654,0.017319,0.006334,-0.006175,0.006953,0.001277,-0.013027
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.003440,0.011453,...,-0.015015,-0.019046,0.015313,0.003978,-0.008323,0.008902,-0.003899,-0.013061,0.011453,0.003440
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.018047,0.037394,...,0.029254,0.015857,0.004441,0.027159,-0.041982,-0.021236,0.005203,-0.024027,0.037394,0.018047
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.084825,-0.026222,...,-0.104377,-0.066035,0.055706,-0.051816,0.080410,0.116916,-0.119805,-0.040657,0.026222,-0.084825
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.043223,0.005021,...,0.026354,-0.048356,0.035233,0.067210,-0.054791,-0.024377,0.024692,-0.018139,-0.005021,0.043223
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.009349,-0.031973,...,0.043557,0.051909,-0.042604,-0.009129,0.022361,-0.026100,0.011385,0.036684,-0.031973,-0.009349
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.028827,0.057672,...,0.053523,0.039631,-0.002767,0.038861,-0.065675,-0.039973,0.010875,-0.033361,0.057672,0.028827


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 175 ms, sys: 5 µs, total: 175 ms
Wall time: 174 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
481666,2675,H_-0.35_-1.9_2.0036.dat,-0.35,-1.90,2.003604,999,1,test,-0.037155,-0.029830,...,-0.025494,0.075339,0.050645,-0.013778,0.015290,0.062937,0.044535,-0.029842,0.029830,0.037155
3117,17,H_-2_-1.15_-1.00676.dat,-2.00,-1.15,-1.006763,999,0,test,-0.005520,-0.083066,...,0.111953,0.036320,-0.068353,0.040995,0.005830,-0.092496,0.060139,0.092247,-0.083066,-0.005520
578288,3212,H_-0.05_0.65_0.000387357.dat,-0.05,0.65,0.000387,0,0,val,0.003333,0.029562,...,-0.073833,-0.083459,0.051665,0.059503,-0.022044,-0.027879,-0.012948,-0.004542,0.029562,0.003333
586971,3260,H_0_-1_1.00325.dat,0.00,-1.00,1.003254,1,1,train,0.028151,0.006932,...,0.028116,-0.048027,-0.021379,0.041455,0.014223,-0.034949,-0.007257,0.028084,-0.006932,-0.028151
829034,4605,H_0.8_1.45_2.00451.dat,0.80,1.45,2.004505,999,1,test,0.056030,-0.009573,...,0.125471,-0.081379,-0.075260,0.068187,0.027661,-0.106507,-0.041157,0.120745,0.009573,-0.056030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482099,2678,H_-0.35_-1.75_2.00376.dat,-0.35,-1.75,2.003757,999,1,test,-0.017703,0.021774,...,-0.066915,0.073848,-0.084209,0.070416,0.060402,-0.017651,0.126794,-0.117973,0.021774,-0.017703
929213,5162,H_1.15_0.95_-1.01112.dat,1.15,0.95,-1.011120,999,1,test,0.060856,-0.030587,...,0.058613,-0.083997,-0.038148,-0.004042,-0.096699,0.078439,-0.083053,0.112789,-0.030587,0.060856
298756,1659,H_-1_-0.05_-0.00239755.dat,-1.00,-0.05,-0.002398,0,0,val,0.108815,-0.074445,...,0.092052,-0.091871,-0.100728,0.045379,0.068831,0.025500,0.005875,-0.092784,-0.074445,0.108815
463572,2575,H_-0.45_1.2_0.000624463.dat,-0.45,1.20,0.000624,0,0,train,-0.001038,0.000597,...,0.000736,-0.003454,0.001893,-0.000242,0.002034,0.001660,0.000545,0.001862,-0.000597,0.001038


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.27251943301326015
% val:  0.27251943301326015
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  1788
number of val hamiltonians:  1788
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [2231, 2923, 3248, 646, 3045, 5834, 3808, 3733, 2561, 5048, 4471, 3129, 4006, 4096, 4179, 1912, 1451, 1838, 6338, 6498, 3163, 2018, 2721, 4539, 307, 684, 1040, 6249, 889, 1006, 1927, 6011, 553, 3004, 5023, 6492, 530, 6408, 3597, 1033, 2072, 6169, 2979, 1044, 1335, 2224, 2681, 5597, 2931, 1779, 3906, 3848, 5364, 2527, 6087, 2818, 4498, 2945, 4627, 1680, 2552, 4406, 1603, 5370, 3501, 3858, 1012, 4054, 5435, 809, 3600, 6493, 4059, 1765, 4719, 5776, 3581, 5022, 1098, 2806, 4658, 3755, 3182, 1767, 6005, 603, 5935, 2689, 1107, 1257, 1046, 4968, 1819, 208, 2479, 1831, 2402, 2310, 1453, 1443, 2604, 2084, 160, 1093, 4333, 3413, 311, 4489, 3756, 554, 2138, 890, 5199, 3054, 1935, 3654, 2973, 2770, 5361, 1750, 1648, 1186, 3497, 787

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 86 µs, sys: 1 µs, total: 87 µs
Wall time: 57.2 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.611111,0.388889,999,-1
1,-2.0,-1.95,test,0.555556,0.444444,999,-1
2,-2.0,-1.90,test,0.516667,0.483333,999,-1
3,-2.0,-1.85,test,0.516667,0.483333,999,-1
4,-2.0,-1.80,test,0.522222,0.477778,999,-1
...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.661111,0.338889,999,-1
6557,2.0,1.85,test,0.644444,0.355556,999,-1
6558,2.0,1.90,test,0.722222,0.277778,999,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1180980


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,0,test
2,0,999,1,test
3,0,999,0,test
4,0,999,1,test
...,...,...,...,...
1180975,6560,999,0,test
1180976,6560,999,0,test
1180977,6560,999,0,test
1180978,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9998912503107134,
 'eigenvector_val': 0.9903243847874721,
 'eigenvector_test': 0.8133169934640523,
 'hamiltonian_train': 0.0,
 'hamiltonian_val': 0.0,
 'hamiltonian_test': 0.004901960784313725}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}